
# Chapter 2: OSTEP 전처리 및 청크 JSON 저장

이 노트북은 OSTEP 교재 PDF를 전처리하고 토큰 기반 청킹 후 JSON으로 저장하는 전체 과정을 단계별로 실습합니다.

## 📚 학습 목표
- PDF → 텍스트 추출 파이프라인 구성 및 품질 관리 포인트 이해
- 목차(챕터/파트/서브섹션) 파싱과 페이지 범위 계산 방법 습득
- 문장 경계 유지형 토큰 기반 청킹과 오버랩 설계 이해

## 📋 실습 구성
- 1️⃣ 환경 설정: 패키지 설치, 경로/상수 정의, 데이터 위치 확인
- 2️⃣ 임포트/상수: 정규식 패턴, 토큰/오버랩, 분리 규칙 등 설정
- 3️⃣ PDF 로드/기본 함수: 페이지 안전 추출 유틸리티 준비
- 4️⃣ 목차 파싱: 챕터/파트/서브섹션 구조화
- 5️⃣ 범위 계산/텍스트 추출: 챕터별 범위 산출 및 정제
- 6️⃣ 청크 생성/저장: 문장 경계 유지형 토큰 청킹 → JSON 저장
- 7️⃣ 결과 요약: 범위/문자수/서브섹션 통계 출력

> ⚠️ 실습 셀 실행 전, 환경 설정 셀(1️⃣)을 먼저 실행하고 OSTEP PDF 경로를 확인하세요.


---
## 1️⃣ Google Colab 환경 설정

이 노트북은 **Google Colab에서 GPU를 사용**하여 실행하도록 설계되었습니다.

### 📌 실행 전 준비사항
1. **런타임 유형 설정**: 메뉴에서 `런타임` → `런타임 유형 변경` → `GPU` 선택
2. **첫 번째 코드 셀 실행**: Google Drive 마운트 및 필수 패키지 자동 설치
3. **OSTEP PDF 업로드**: 최초 1회만 `/content/drive/MyDrive/ostep_rag/data/documents/ostep.pdf` 위치에 업로드

> ⚠️ **중요**: 아래 코드 셀을 가장 먼저 실행하여 환경을 설정하세요.


In [1]:
# ========================================
# Google Colab 환경 설정
# ========================================
from google.colab import drive
import os

# Google Drive 마운트
drive.mount('/content/drive')

# 필요 패키지 설치
!pip -q install PyPDF2 pdfplumber pymupdf tiktoken

# 경로 설정 (Colab 전용)
BASE_DIR = "/content/drive/MyDrive/ostep_rag"
DATA_DIR = os.path.join(BASE_DIR, "data")
DOC_ID = "ostep"
OUTPUT_DIR = os.path.join(DATA_DIR, "chunk")
PDF_PATH = os.path.join(DATA_DIR, "documents", "ostep.pdf")

# 디렉토리 생성
os.makedirs(os.path.join(DATA_DIR, "documents"), exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# OSTEP PDF 업로드 확인
if not os.path.exists(PDF_PATH):
    print(f"⚠️ 구글 드라이브에 OSTEP PDF를 업로드하세요:")
    print(f"   {PDF_PATH}")
    print(f"\n📁 좌측 파일 탭 → drive → MyDrive → ostep_rag → data → documents 폴더에 ostep.pdf 업로드")
else:
    print(f"✅ OSTEP PDF 파일 확인됨: {PDF_PATH}")

ModuleNotFoundError: No module named 'google.colab'


---
## 2️⃣ 임포트 및 설정 상수

이 셀에서는 필요한 라이브러리를 임포트하고 전역 설정 상수를 정의합니다.

**주요 내용:**
- PyPDF2를 사용하여 PDF 파일 처리
- 정규식 패턴을 사용하여 목차에서 챕터, 파트, 서브섹션 파싱
- 토큰 기반 청킹 설정(최대 토큰, 오버랩 토큰)
- 출력 디렉토리 및 문서 ID 설정


In [ ]:
import re
import os
import json
from pathlib import Path
from PyPDF2 import PdfReader
import warnings
warnings.filterwarnings('ignore')

# 설정 상수
TOC_START_PAGE = 15
TOC_END_PAGE = 25
PAGE_OFFSET = 36
CHUNK_MAX_TOKENS = 400
CHUNK_OVERLAP_TOKENS = 80
SENTENCE_SPLIT_REGEX = r'(?<=[.!?])\s+(?=[A-Z0-9])'

# 정규식 패턴
MAIN_CHAPTER_PATTERNS = [
    r'^(\d+)\s+(.+?)\s+\.{3,}\s*(\d+)$',
    r'^(\d+)\s+(.+?)\s+(\d+)$'
]

PART_PATTERNS = [
    r'^([IVX]+)\s+(.+?)\s+\.{3,}\s*(\d+)$',
    r'^([IVX]+)\s+(.+?)\s+(\d+)$'
]

SUBSECTION_PATTERNS = [
    r'^(\d+\.\d+)\s+(.+?)\s+\.{3,}\s*(\d+)$',
    r'^(\d+\.\d+)\s+(.+?)\s+(\d+)$'
]

---
## 3️⃣ PDF 로드 및 기본 함수

이 셀에서는 PDF 파일을 로드하고 기본적인 텍스트 추출 함수를 정의합니다.

**주요 함수:**
- `load_pdf()`: PDF 파일을 로드하고 페이지 수를 확인
- `extract_pdf_text()`: 지정된 페이지 범위에서 텍스트를 추출

**실행 결과:**
- PDF 파일이 성공적으로 로드되고 전체 페이지 수가 출력됩니다.


In [ ]:
def load_pdf(pdf_path: str):
    """PDF 파일 로드"""
    pdf_path = Path(pdf_path)
    if not pdf_path.exists():
        raise FileNotFoundError(f"PDF 파일을 찾을 수 없습니다: {pdf_path}")
    
    reader = PdfReader(pdf_path)
    print(f"PDF 로드 완료: {len(reader.pages)} 페이지")
    return reader

def extract_pdf_text(reader, start_page: int, end_page: int):
    """PDF 페이지 범위에서 텍스트 추출"""
    total_pages = len(reader.pages)
    if start_page < 1 or end_page > total_pages:
        start_page = max(1, start_page)
        end_page = min(total_pages, end_page)
    
    text = ""
    for page_num in range(start_page - 1, end_page):
        page = reader.pages[page_num]
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    
    return text.strip()

# PDF 로드 실행
pdf_path = PDF_PATH
reader = load_pdf(PDF_PATH)

PDF 로드 완료: 643 페이지


---
## 4️⃣ 목차 파싱

이 셀에서는 PDF 목차를 파싱하여 챕터, 파트, 서브섹션 정보를 추출합니다.

**주요 함수:**
- `create_toc_entry()`: 목차 항목을 딕셔너리로 생성
- `parse_chapter_line()`, `parse_part_line()`, `parse_subsection_line()`: 각 유형별 라인 파싱
- `parse_toc()`: 목차 전체를 파싱하여 3가지 유형으로 분류

**실행 결과:**
- 챕터, 파트, 서브섹션의 개수가 출력됩니다.
- 정규식 패턴으로 목차의 구조를 인식하여 계층 구조를 파악합니다.


In [ ]:
def create_toc_entry(entry_type: str, number, title: str, toc_page: int):
    """TOC 항목 생성 (딕셔너리 반환)"""
    actual_page = toc_page + PAGE_OFFSET
    
    if entry_type == 'part':
        full_title = f"{number} {title}"
        entry_id = f"part_{number}"
    elif entry_type == 'subsection':
        full_title = f"{number} {title}"
        entry_id = f"subsec_{number}"
    else:  # chapter
        full_title = f"{number} {title}"
        entry_id = f"ch_{number}"
    
    return {
        'entry_type': entry_type,
        'number': number,
        'title': title,
        'toc_page': toc_page,
        'actual_page': actual_page,
        'full_title': full_title,
        'id': entry_id
    }

def clean_toc_title(title: str):
    """목차 제목 정제"""
    title = re.sub(r'\s*[\.\s]{4,}.*$', '', title)
    title = re.sub(r'\s*\.{3,}.*$', '', title)
    title = re.sub(r'\s*\.+\s*$', '', title)
    title = re.sub(r'\s+', ' ', title)
    return title.strip()

def is_valid_title(title: str):
    """제목 유효성 검사"""
    return (bool(title) and 
            len(title) > 3 and 
            not title.strip().isdigit() and 
            title[0].isupper())

def parse_chapter_line(line: str):
    """챕터 라인 파싱"""
    for pattern in MAIN_CHAPTER_PATTERNS:
        match = re.match(pattern, line)
        if match:
            chapter_num = int(match.group(1))
            title = clean_toc_title(match.group(2).strip())
            page_num = int(match.group(3))
            
            if is_valid_title(title):
                return create_toc_entry('chapter', chapter_num, title, page_num)
    return None

def parse_part_line(line: str):
    """파트 라인 파싱"""
    for pattern in PART_PATTERNS:
        match = re.match(pattern, line)
        if match:
            part_num = match.group(1)
            title = clean_toc_title(match.group(2).strip())
            page_num = int(match.group(3))
            
            if is_valid_title(title):
                return create_toc_entry('part', part_num, title, page_num)
    return None

def parse_subsection_line(line: str):
    """서브섹션 라인 파싱 (예: 2.1, 4.2 등)"""
    for pattern in SUBSECTION_PATTERNS:
        match = re.match(pattern, line)
        if match:
            subsection_num = match.group(1)
            title = clean_toc_title(match.group(2).strip())
            page_num = int(match.group(3))
            
            if is_valid_title(title):
                return create_toc_entry('subsection', subsection_num, title, page_num)
    return None

def parse_toc(reader):
    """목차 파싱 (챕터 + 파트 + 서브섹션)"""
    print(f"목차 파싱 중 (페이지 {TOC_START_PAGE}-{TOC_END_PAGE})...")
    
    toc_text = extract_pdf_text(reader, TOC_START_PAGE, TOC_END_PAGE)
    if not toc_text:
        print("목차 텍스트 추출 실패")
        return [], [], []
    
    chapters = []
    parts = []
    subsections = []
    
    for line in toc_text.split('\n'):
        line = line.strip()
        if not line:
            continue
        
        # 파트 파싱 우선 시도
        part_entry = parse_part_line(line)
        if part_entry:
            parts.append(part_entry)
            continue
        
        # 서브섹션 파싱 시도
        subsection_entry = parse_subsection_line(line)
        if subsection_entry:
            subsections.append(subsection_entry)
            continue
        
        # 챕터 파싱
        chapter_entry = parse_chapter_line(line)
        if chapter_entry:
            chapters.append(chapter_entry)
    
    # 페이지 순서로 정렬
    chapters.sort(key=lambda x: x['toc_page'])
    parts.sort(key=lambda x: x['toc_page'])
    subsections.sort(key=lambda x: x['toc_page'])
    
    print(f"목차 파싱 완료: {len(chapters)}개 챕터, {len(parts)}개 파트, {len(subsections)}개 서브섹션")
    return chapters, parts, subsections

# 목차 파싱 실행
chapters, parts, subsections = parse_toc(reader)


목차 파싱 중 (페이지 15-25)...
목차 파싱 완료: 50개 챕터, 3개 파트, 297개 서브섹션


---
## 5️⃣ 챕터 범위 계산

이 셀에서는 각 챕터가 차지하는 페이지 범위를 계산합니다.

**주요 함수:**
- `find_part_for_chapter()`: 챕터가 속한 파트를 찾기
- `calculate_chapter_ranges()`: 챕터별 시작/종료 페이지를 계산

**실행 결과:**
- 각 챕터의 페이지 범위가 계산됩니다.
- 다음 챕터 시작 전까지가 현재 챕터의 마지막 페이지입니다.


In [ ]:
def find_part_for_chapter(chapter, parts):
    """챕터가 속한 파트 찾기"""
    current_part = None
    for part in parts:
        if part['actual_page'] <= chapter['actual_page']:
            current_part = part
        else:
            break
    return current_part

def calculate_chapter_ranges(reader, chapters, parts):
    """챕터별 페이지 범위 계산"""
    print("챕터별 페이지 범위 계산 중...")
    
    if not chapters:
        print("메인 챕터를 찾을 수 없습니다.")
        return []
    
    chapter_ranges = []
    total_pages = len(reader.pages)
    
    for i, chapter in enumerate(chapters):
        start_page = chapter['actual_page']
        
        # 다음 챕터가 있으면 그 전 페이지까지, 없으면 PDF 끝까지
        if i + 1 < len(chapters):
            end_page = chapters[i + 1]['actual_page'] - 1
        else:
            end_page = total_pages
        
        # 해당 챕터가 속한 파트 찾기
        part_info = find_part_for_chapter(chapter, parts)
        
        # 페이지 범위 유효성 검사
        if start_page <= end_page:
            chapter_range = {
                'toc_entry': chapter,
                'part_info': part_info,
                'start_page': start_page,
                'end_page': end_page,
                'page_count': end_page - start_page + 1
            }
            chapter_ranges.append(chapter_range)
    
    print(f"챕터 범위 계산 완료: {len(chapter_ranges)}개 챕터")
    return chapter_ranges

# 챕터 범위 계산 실행
chapter_ranges = calculate_chapter_ranges(reader, chapters, parts)


챕터별 페이지 범위 계산 중...
챕터 범위 계산 완료: 50개 챕터


---
## 6️⃣ 텍스트 추출 및 서브섹션 분할

이 셀에서는 각 챕터의 텍스트를 추출하고 서브섹션으로 분할합니다.

**주요 함수:**
- `clean_text()`: PDF 헤더/푸터, 페이지 번호 제거 및 공백 정리
- `find_chapter_subsections()`: 챕터에 속한 서브섹션들 찾기
- `split_text_by_subsections()`: 텍스트를 서브섹션별로 분할
- `extract_texts()`: 모든 챕터에서 텍스트 추출 및 서브섹션 분할

**실행 결과:**
- 각 챕터의 텍스트가 추출되고 서브섹션별로 분할됩니다.
- 헤더/푸터 등 불필요한 요소가 제거된 깨끗한 텍스트를 얻습니다.


In [ ]:
def clean_text(text: str):
    """텍스트 정제 및 헤더/푸터 제거"""
    # PDF 헤더/푸터 제거
    header_pattern = r'c/circle\s*copyrt\s*\d+,?\s*A\s+RPACI\s*-?\s*D\s*USSEAU\s*THREE\s+EASY\s+PIECES'
    text = re.sub(header_pattern, '', text, flags=re.IGNORECASE)
    
    footer_pattern = r'OPERATING\s+SYSTEMS\s+\[VERSION\s+[\d.]+\]\s+WWW\s*\.\s*OSTEP\s*\.\s*ORG.*$'
    text = re.sub(footer_pattern, '', text, flags=re.IGNORECASE | re.MULTILINE)
    
    # 페이지 번호 패턴 제거
    text = re.sub(r'^\s*\d+\s*$', '', text, flags=re.MULTILINE)
    
    # 공백 정리
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def find_chapter_subsections(chapter, subsections):
    """챕터에 속한 서브섹션들 찾기"""
    chapter_num = str(chapter['number'])
    chapter_subsections = []
    
    for subsection in subsections:
        # 서브섹션 번호가 해당 챕터로 시작하는지 확인 (예: 2.1, 2.2는 챕터 2에 속함)
        subsection_num = str(subsection['number'])
        if subsection_num.startswith(f"{chapter_num}."):
            chapter_subsections.append(subsection)
    
    # 번호 순으로 정렬
    chapter_subsections.sort(key=lambda x: float(x['number']))
    return chapter_subsections

def build_flexible_title_pattern(title: str):
    """제목을 안전한 정규식으로 변환"""
    parts = []
    for ch in title:
        if ch.isspace():
            parts.append(r"\s+")
        elif ch == '(':
            parts.append(r"\s*\(\s*")
        elif ch == ')':
            parts.append(r"\s*\)\s*")
        else:
            parts.append(re.escape(ch))
    return ''.join(parts)

def split_text_by_subsections(chapter_text: str, chapter, chapter_subsections):
    """챕터 텍스트를 서브섹션별로 분할"""
    if not chapter_subsections:
        return []
    
    subsection_data_list = []
    
    split_points = []
    
    for subsection in chapter_subsections:
        section_title = f"{subsection['number']} {subsection['title']}"
        
        patterns_to_try = [
            re.escape(section_title),
            f"{re.escape(str(subsection['number']))}\\s+{build_flexible_title_pattern(subsection['title'])}"
        ]

        # "The" 시작하는 제목의 특수 처리
        lower_title = subsection['title'].lower()
        if lower_title.startswith("the "):
            after_the = subsection['title'][4:]
            patterns_to_try.append(
                f"{re.escape(str(subsection['number']))}\\s+the\\s*{build_flexible_title_pattern(after_the)}"
            )
        elif lower_title.startswith("the"):
            after_the = subsection['title'][3:]
            patterns_to_try.append(
                f"{re.escape(str(subsection['number']))}\\s+the\\s*{build_flexible_title_pattern(after_the)}"
            )
        
        found = False
        for pattern in patterns_to_try:
            matches = list(re.finditer(pattern, chapter_text, re.IGNORECASE | re.MULTILINE))
            if matches:
                # 첫 번째 매치 사용
                match = matches[0]
                split_points.append({
                    'subsection': subsection,
                    'start_pos': match.start(),
                    'end_pos': match.end(),
                    'title_match': match.group().strip()
                })
                found = True
                break
        
        if not found:
            print(f"  서브섹션 '{subsection['number']} {subsection['title']}' 제목을 텍스트에서 찾을 수 없습니다.")
    
    # 위치별로 정렬
    split_points.sort(key=lambda x: x['start_pos'])
    
    # 텍스트 분할
    for i, split_point in enumerate(split_points):
        start_pos = split_point['start_pos']
        
        # 다음 서브섹션의 시작 위치 또는 텍스트 끝
        if i + 1 < len(split_points):
            end_pos = split_points[i + 1]['start_pos']
        else:
            end_pos = len(chapter_text)
        
        # 서브섹션 텍스트 추출
        subsection_text = chapter_text[start_pos:end_pos].strip()
        
        if subsection_text:
            subsection_data = {
                'toc_entry': split_point['subsection'],
                'parent_chapter': chapter,
                'text': clean_text(subsection_text)
            }
            subsection_data_list.append(subsection_data)
    
    return subsection_data_list

def extract_texts(reader, chapter_ranges, subsections):
    """챕터별 텍스트 추출 및 서브섹션 분할"""
    print(f"챕터별 텍스트 추출 및 서브섹션 분할 중...")
    for chapter_range in chapter_ranges:
        chapter = chapter_range['toc_entry']
        
        # 텍스트 추출
        text = extract_pdf_text(reader, chapter_range['start_page'], chapter_range['end_page'])
        chapter_range['text'] = clean_text(text)
        
        # 해당 챕터의 서브섹션들 찾기
        chapter_subsections = find_chapter_subsections(chapter, subsections)
        
        if chapter_subsections:
            # 서브섹션별로 텍스트 분할
            subsection_data_list = split_text_by_subsections(chapter_range['text'], chapter, chapter_subsections)
            chapter_range['subsections'] = subsection_data_list
        else:
            chapter_range['subsections'] = []
    
    print(f"챕터별 텍스트 추출 및 서브섹션 분할 완료: {len(chapter_ranges)}개 챕터")
    return chapter_ranges

# 텍스트 추출 및 서브섹션 분할 실행
chapter_ranges = extract_texts(reader, chapter_ranges, subsections)


챕터별 텍스트 추출 및 서브섹션 분할 중...
챕터별 텍스트 추출 및 서브섹션 분할 완료: 50개 챕터


---
## 7️⃣ 청크 생성 및 JSON 저장

이 셀에서는 텍스트를 토큰 기반으로 청킹하고 JSON 파일로 저장합니다.

**주요 함수:**
- `estimate_tokens_length()`: tiktoken 또는 근사치로 토큰 수 추정
- `split_text_into_sentences()`: 정규식으로 문장 분리
- `chunk_sentences_hybrid()`: 문장 경계를 유지하며 토큰 제한으로 청킹
- `build_all_chunk_records()`: 모든 청크를 레코드로 생성
- `write_json()`: JSON 파일로 저장

**실행 결과:**
- 문장 경계를 유지하면서 토큰 제한(400토큰, 20% 오버랩)에 맞춰 청킹됩니다.
- JSON 파일로 저장되어 RAG 시스템에서 사용할 수 있습니다.


In [ ]:
def ensure_output_dir(output_dir: str):
    os.makedirs(output_dir, exist_ok=True)

def build_output_path(output_dir: str, max_tokens: int, overlap_tokens: int):
    overlap_pct = int(round((overlap_tokens / max_tokens) * 100)) if max_tokens > 0 else 0
    filename = f"{DOC_ID}_tok{max_tokens}_ov{overlap_pct}.json"
    return str(Path(output_dir) / filename)

def try_import_tiktoken():
    try:
        import tiktoken
        return tiktoken
    except Exception:
        return None

def estimate_tokens_length(text: str):
    """토큰 길이 추정 (tiktoken 우선, 없으면 근사치)"""
    tiktoken = try_import_tiktoken()
    if tiktoken is not None:
        try:
            enc = tiktoken.get_encoding("cl100k_base")
        except Exception:
            enc = tiktoken.get_encoding("cl100k_base")
        return len(enc.encode(text))
    return int(len(text.split()) * 1.3)

def split_text_into_sentences(text: str):
    """정규식 기반 문장 분리"""
    text = text.strip()
    if not text:
        return []
    sentences = re.split(SENTENCE_SPLIT_REGEX, text)
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences

def chunk_sentences_hybrid(sentences, max_tokens: int, overlap_tokens: int):
    """문장 경계를 유지하며 토큰 상한으로 청킹"""
    chunks = []
    if not sentences:
        return chunks

    current = []
    current_tokens = 0
    for sent in sentences:
        sent_tokens = estimate_tokens_length(sent)
        
        if sent_tokens > max_tokens:
            if current:
                chunks.append(" ".join(current).strip())
                current = []
                current_tokens = 0
            chunks.append(sent.strip())
            continue

        if current_tokens + sent_tokens <= max_tokens:
            current.append(sent)
            current_tokens += sent_tokens
        else:
            if current:
                chunks.append(" ".join(current).strip())
            
            if overlap_tokens > 0:
                overlap_bucket = []
                overlap_count = 0
                for prev_sent in reversed(current):
                    t = estimate_tokens_length(prev_sent)
                    if overlap_count + t > overlap_tokens:
                        break
                    overlap_bucket.append(prev_sent)
                    overlap_count += t
                overlap_bucket.reverse()
                current = overlap_bucket + [sent]
                current_tokens = sum(estimate_tokens_length(s) for s in current)
            else:
                current = [sent]
                current_tokens = sent_tokens

    if current:
        chunks.append(" ".join(current).strip())
    return chunks

def make_chunk_id(chapter_id: str, subsec_id, chunk_index: int):
    base = f"{chapter_id}"
    if subsec_id:
        base += f"__{subsec_id}"
    return f"{base}__{chunk_index:04d}"

def find_part_for_page(page: int, part_lookup_by_page):
	current = None
	for start_page, part in part_lookup_by_page:
		if start_page <= page:
			current = part
		else:
			break
	return current

def build_all_chunk_records(pdf_path: str, parts, chapter_ranges, max_tokens: int, overlap_tokens: int):
    part_lookup_by_page = []
    for p in parts:
        part_lookup_by_page.append((p['actual_page'], p))
    part_lookup_by_page.sort(key=lambda x: x[0])

    records = []

    for cr in chapter_ranges:
        chapter = cr['toc_entry']
        chapter_id = chapter['id']
        chapter_title = chapter['title']
        part_info = cr.get('part_info')
        if part_info is None:
            part_info = find_part_for_page(cr['start_page'], part_lookup_by_page)

        # 서브섹션이 있으면 서브섹션 기준, 없으면 챕터 전체 텍스트 기준으로 청킹
        if cr.get('subsections'):
            for sub in cr['subsections']:
                subsec = sub
                sentences = split_text_into_sentences(subsec['text'])
                chunks = chunk_sentences_hybrid(sentences, max_tokens, overlap_tokens)
                for idx, chunk_text in enumerate(chunks):
                    records.append({
                        'chunk_id': make_chunk_id(chapter_id, subsec['toc_entry']['id'], idx),
                        'chapter_id': chapter_id,
                        'chapter_title': chapter_title,
                        'subsection_id': subsec['toc_entry']['id'],
                        'subsection_title': f"{subsec['toc_entry']['number']} {subsec['toc_entry']['title']}",
                        'text': chunk_text,
                    })
        else:
            sentences = split_text_into_sentences(cr['text'])
            chunks = chunk_sentences_hybrid(sentences, max_tokens, overlap_tokens)
            for idx, chunk_text in enumerate(chunks):
                records.append({
                    'chunk_id': make_chunk_id(chapter_id, None, idx),
                    'chapter_id': chapter_id,
                    'chapter_title': chapter_title,
                    'subsection_id': None,
                    'subsection_title': None,
                    'text': chunk_text,
                })

    return records

def write_json(path: str, records):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(records, f, ensure_ascii=False, indent=2)

# 청크 생성 및 저장
print("\n5단계: 청크 생성 및 저장(JSON)")
print("-" * 40)
ensure_output_dir(OUTPUT_DIR)
output_path = build_output_path(OUTPUT_DIR, CHUNK_MAX_TOKENS, CHUNK_OVERLAP_TOKENS)
all_chunk_records = build_all_chunk_records(
	pdf_path=pdf_path,
	parts=parts,
	chapter_ranges=chapter_ranges,
	max_tokens=CHUNK_MAX_TOKENS,
	overlap_tokens=CHUNK_OVERLAP_TOKENS,
)
write_json(output_path, all_chunk_records)
print(f"청크 저장 완료: {output_path} ({len(all_chunk_records)} chunks)")



5단계: 청크 생성 및 저장(JSON)
----------------------------------------
청크 저장 완료: ../data/chunk/ostep_tok400_ov20.json (991 chunks)


---
## 8️⃣ 결과 요약 출력

이 셀에서는 전처리된 데이터의 요약 정보를 출력합니다.

**주요 내용:**
- 챕터별 페이지 범위, 문자 수, 서브섹션 개수
- 서브섹션별 문자 수

**실행 결과:**
- 각 챕터와 서브섹션의 상세 정보가 출력됩니다.
- 전처리 품질을 확인하고 개선점을 파악할 수 있습니다.


In [ ]:
# 결과 요약
total_subsections = sum(len(cr.get('subsections', [])) for cr in chapter_ranges)

print("\nChapter information:")
for info in chapter_ranges:
    part_str = f"Part {info['part_info']['number']}" if info['part_info'] else 'Unclassified'
    print(f"  • {info['toc_entry']['full_title']} ({part_str}): {info['start_page']}-{info['end_page']} "
        f"({info['page_count']}p, {len(info['text']):,} chars, {len(info.get('subsections', []))} subsections)")
    
    # 서브섹션 정보 출력
    if info.get('subsections'):
        for subsection in info['subsections']:
            print(f"    ├─ {subsection['toc_entry']['number']} {subsection['toc_entry']['title']} ({len(subsection['text']):,} chars)")



Chapter information:
  • 1 A Dialogue on the Book (Unclassified): 37-38 (2p, 3,519 chars, 0 subsections)
  • 2 Introduction to Operating Systems (Unclassified): 39-58 (20p, 44,104 chars, 7 subsections)
    ├─ 2.1 Virtualizing the CPU (3,750 chars)
    ├─ 2.2 Virtualizing Memory (3,052 chars)
    ├─ 2.3 Concurrency (5,386 chars)
    ├─ 2.4 Persistence (4,965 chars)
    ├─ 2.5 Design Goals (3,919 chars)
    ├─ 2.6 Some History (11,916 chars)
    ├─ 2.7 Summary (5,596 chars)
  • 3 A Dialogue on Virtualization (Part I): 59-60 (2p, 2,285 chars, 0 subsections)
  • 4 The Abstraction: The Process (Part I): 61-70 (10p, 18,608 chars, 6 subsections)
    ├─ 4.1 The Abstraction: A Process (2,252 chars)
    ├─ 4.2 Process API (1,628 chars)
    ├─ 4.3 Process Creation: A Little More Detail (3,449 chars)
    ├─ 4.4 Process States (1,697 chars)
    ├─ 4.5 Data Structures (4,060 chars)
    ├─ 4.6 Summary (2,061 chars)
  • 5 Interlude: Process API (Part I): 71-80 (10p, 19,141 chars, 6 subsections)
    ├